In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) #  os.path.join( os.environ['HOME'],'Code'), 'ImageD11' )

In [ ]:
import numba
import numpy as np
import scipy.ndimage as ndi
from matplotlib import pyplot as plt

from ImageD11.grain import grain
from ImageD11 import unitcell
import ImageD11.sinograms.dataset
from ImageD11.sinograms.point_by_point import PBPMap, nb_inv
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.nbGui import nb_utils as utils

%matplotlib ipympl

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)

sample = ds.sample
dataset = ds.dsname
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# define your reference unit cell for RGB colour plotting

cf_2d = ds.get_cf_2d_from_disk()
cf_2d.parameters.loadparameters(ds.parfile)

cf_pars = cf_2d.parameters.get_parameters()

ref_ucell = ImageD11.unitcell.unitcell_from_parameters(cf_pars)

In [ ]:
pbpfile = os.path.join(ds.analysispath, ds.dsname + '_pbp.txt')

pmap = PBPMap(pbpfile)

In [ ]:
fig, ax = plt.subplots()
ax.hist(pmap.u, bins=np.arange(0.5,np.max(pmap.u)+0.51,1))
ax.set_xlabel('Unique spots per pixel')
ax.set_ylabel('Count')
plt.show()

In [ ]:
# choose the minimum number of peaks you want a pixel to have to be counted

min_unique = 30

pmap.choose_best(min_unique)

In [ ]:
# let's plot the result of your choice

fig, axs = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(10, 5), constrained_layout=True)
r = np.where(pmap.nuniq > min_unique, pmap.nuniq, 0)
axs[0].imshow(pmap.nuniq, origin="lower")
axs[0].set_title("All pixels")
axs[1].imshow(r, origin="lower")
axs[1].set_title("Best pixels")
plt.show()

In [ ]:
# now make a TensorMap from it so we can plot and export

# first let's work out what phase we have
phases = {0: ref_ucell}

# let's make a phase id map from our pbpmap
phase_ids = TensorMap.recon_order_to_map_order(np.where(pmap.nuniq > min_unique, 0, -1))

tmap = TensorMap.from_pbpmap(pmap, steps=(1, ds.ystep, ds.ystep), phases=phases)
tmap['phase_ids'] = phase_ids

In [ ]:
tmap.plot('nuniq')

In [ ]:
# compute the IPF colours from the UBIs and phase

tmap.get_ipf_maps()

In [ ]:
tmap.plot('ipf_x')
tmap.plot('ipf_y')
tmap.plot('ipf_z')

In [ ]:
utils.plot_all_ipfs_from_meta_orien(tmap.get_meta_orix_orien(), tmap.phases[0])

In [ ]:
# save the TensorMap to disk

tmap.to_h5(os.path.join(ds.analysispath, 'pbp_tensormap.h5'))
tmap.to_paraview(os.path.join(ds.analysispath, 'pbp_tensormap.h5'))